In [111]:
import pandas as pd
import numpy as np
import re

from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, STAMEN_TONER, CARTODBPOSITRON_RETINA
from bokeh.io import output_notebook, show

### For some reason bokeh maps uses Web Mercator coordinates

In [10]:
# Define function to switch from lat/long to mercator x/y coordinates
def to_mercator(lat, lon):
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [50]:
# Ames, Iowa lat long, converted to mercator
Ames_center = to_mercator(42.034534, -93.620369)

In [76]:
geo = pd.read_csv('geodata.csv')
geo.shape

(2624, 2)

In [77]:
#list of house mercator coordinates
x_merc = []
y_merc = []

for each in geo['lat_long']:
    ll = re.sub("[^0-9.\- ]","",each).split()
    #if 0, append center of Ames (-10421771.804958373, 5166153.874296733)
    if ll == ['0']:
        x_merc.append(Ames_center[0])
        y_merc.append(Ames_center[1])
    else:
        x, y = to_mercator(float(ll[0]),float(ll[1]))
        x_merc.append(x)
        y_merc.append(y)

In [78]:
# concat to geo DataFrame
len(x_merc)

2624

In [82]:
geo = pd.concat([geo, pd.Series(x_merc), pd.Series(y_merc)],axis=1).set_index('PID')

In [85]:
geo = geo.rename(columns={0:'x_merc',1:'y_merc'})

In [112]:
background = get_provider(CARTODBPOSITRON_RETINA) #CARTODBPOSITRON_RETINA, STAMEN_TONER
x_zoom = 7000
y_zoom = 5000

fig = figure(plot_width=1200, plot_height=800,
             x_range=(Ames_center[0]-x_zoom, Ames_center[0]+y_zoom), 
             y_range=(Ames_center[1]-x_zoom, Ames_center[1]+y_zoom),
             x_axis_type="mercator", y_axis_type="mercator")

fig.circle(x="x_merc", y="y_merc",
         size=4,
         fill_color="orange", line_color='dodgerblue',
         fill_alpha=0.6,
         source=geo)

fig.add_tile(background)

show(fig)